In [1]:
using Flux
using Flux.Data: DataLoader
using PyCall
using Plots
using PlotThemes
using TOML
using Pkg
using BSON: @save

In [2]:
env_path = TOML.parse(read("mltools.toml", String))["env_path"]
Pkg.activate(env_path)

  Activating environment at `C:\Users\nikhi\OneDrive\Documents\julia\MLTools\Project.toml`


In [3]:
import MLTools as mlt

┌ Info: Precompiling MLTools [1eca4633-f73c-4459-9b1e-59996da1b775]
└ @ Base loading.jl:1342


In [4]:
df = mlt.getdf(joinpath(@__DIR__, "filtered_dataset.csv"))

,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=C(C=C3)B...,Active
1,COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=CC=C3)N)...,Active
2,C1CN(CCN1)C(=O)C2=C(SC(=C2)C3=CC=CC=C3)N,Active
3,C1=CC=C(C=C1)C(=O)C2=C(C(=C(N2)N)C(=O)N)C3=CC=...,Active
4,CC1=C(C(=O)OC(C1)[C@@H](C)C2CC[C@@H]3[C@@]2(CC...,Active
...,...,...
18229,CCCOC1=C(C=C(C=C1)/C=C/C(=O)OCC(=O)N(C)C2=C(N(...,Inactive
18230,CC(C)NC1=NN=C(S1)SCC(=O)NC2=CC(=C(C(=C2)OC)OC)OC,Inactive
18231,CC1=NN(C2=C1C=C(C=N2)C(=O)C3=C(C=CC(=C3)F)O)CCO,Inactive
18232,CC1=CSC(=N1)NC(=O)CC2=CC(=C(C=C2)OC)S(=O)(=O)N...,Inactive


In [6]:
smiles = mlt.dfToStringMatrix(df)

18234×2 Matrix{String}:
 "COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=C(C=C3)Br)N)C(=O)C4=CC=CC=C4)OC"                                                         …  "Active"
 "COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=CC=C3)N)C(=O)C4=CC=CC=C4)OC"                                                                "Active"
 "C1CN(CCN1)C(=O)C2=C(SC(=C2)C3=CC=CC=C3)N"                                                                                         "Active"
 "C1=CC=C(C=C1)C(=O)C2=C(C(=C(N2)N)C(=O)N)C3=CC=CC=C3Br"                                                                            "Active"
 "CC1=C(C(=O)OC(C1)[C@@H](C)C2CC[C@@H]3[C@@]2(CCC4C3C[C@@H]5[C@]6([C@@]4(C(=O)C=C[C@@H]6O)C)O5)C)CO"                                "Active"
 "CC1CCC/C=C/C(=O)C2=C(CC(=O)O1)C=C(C=C2O)O"                                                                                     …  "Active"
 "CC1C2C(NC(=O)C23C(/C=C(/CCC(C(/C=C/C3=O)O)O)\\C)C=C1C)CC(C)C"                                                                   

In [17]:
vocab = mlt.dfToStringMatrix(mlt.getdf(joinpath(@__DIR__, "vocab.csv")))

71×1 Matrix{String}:
 "#"
 "("
 ")"
 "."
 "/"
 "1"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 ⋮
 "[Nd+3]"
 "[O-]"
 "[OH-]"
 "[P+]"
 "[Pt+2]"
 "[Pt]"
 "[S+]"
 "[S-]"
 "[Se]"
 "[Si]"
 "[Zn+2]"
 "\\"

In [18]:
tokenizer = Dict(j => i for (i, j) in enumerate(vocab))

Dict{String, Int64} with 71 entries:
  "1"      => 6
  "[N@@+]" => 52
  "[H+]"   => 42
  "C"      => 18
  "P"      => 24
  "2"      => 7
  "6"      => 11
  "[C-]"   => 32
  "[C@@H]" => 33
  "[Hg]"   => 44
  "[S-]"   => 67
  "O"      => 23
  "[Co+3]" => 40
  "B"      => 16
  "="      => 15
  "/"      => 5
  "."      => 4
  "Cl"     => 19
  "[Pt+2]" => 64
  "[C@@]"  => 34
  "[Ca+2]" => 38
  "[Pt]"   => 65
  "7"      => 12
  "[NH-]"  => 54
  "#"      => 1
  ⋮        => ⋮

In [ ]:
function readPyFile(path)
    py"""
    exec(open($(path)).read(), globals(), locals())
    """
end

readPyFile("tokenization.py")
return_tokens(str) = py"return_tokens"(str)
# test output